In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

import os

In [3]:
df = pd.read_csv("data/Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [5]:
df.describe(percentiles=[0.25, 0.5,0.75])

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [6]:
df.drop(["RowNumber", "CustomerId", "Surname"], axis=1, inplace=True)

In [7]:
df.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
#train test split 
X_train, X_test, y_train, y_test = train_test_split(df.drop("Exited", axis=1), df["Exited"], test_size=0.2, random_state=42)

In [9]:
X_train["Gender"].value_counts()

Gender
Male      4362
Female    3638
Name: count, dtype: int64

In [10]:
X_train.to_csv("data/X_train.csv", index=False)
X_test.to_csv("data/X_test.csv", index=False)
y_train.to_csv("data/y_train.csv", index=False)
y_test.to_csv("data/y_test.csv", index=False)

In [11]:
# Label Encoding for Gender Column
lablecoder = LabelEncoder()
X_train["Gender"] = lablecoder.fit_transform(X_train["Gender"])
X_test["Gender"] = lablecoder.transform(X_test["Gender"])

In [ ]:
# One-Hot Encoding for Geography Column
onecoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', dtype=np.int64)
X_train_geo = onecoder.fit_transform(X_train["Geography"].values.reshape(-1, 1))
X_test_geo = onecoder.transform(X_test["Geography"].values.reshape(-1, 1))

In [14]:
# Combine the encoded features with the original dataset (excluding the original "Geography" column)
x_train_encpd = pd.DataFrame(X_train_geo, columns=onecoder.get_feature_names_out(["Geography"]))
X_train = pd.concat([X_train.drop("Geography", axis=1).reset_index(drop=True), x_train_encpd], axis=1)

x_test_encpd = pd.DataFrame(X_test_geo, columns=onecoder.get_feature_names_out(["Geography"]))
X_test = pd.concat([X_test.drop("Geography", axis=1).reset_index(drop=True), x_test_encpd], axis=1)



In [15]:
# save the processed data and encoders
os.makedirs("artifacts", exist_ok=True)
with open("artifacts/onehotencoder_geography.pkl", "wb") as file:
    pickle.dump(onecoder, file)

with open("artifacts/labelencoder_gender.pkl", "wb") as file:
    pickle.dump(lablecoder, file)

In [16]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [17]:
# save the scaler
with open("artifacts/standardscaler.pkl", "wb") as file:
    pickle.dump(scaler, file)

In [18]:
X_test

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,596,1,32,3,96709.07,2,0,0,41788.37,0,1,0
1,623,1,43,1,0.00,2,1,1,146379.30,1,0,0
2,601,0,44,4,0.00,2,1,0,58561.31,0,0,1
3,506,1,59,8,119152.10,2,1,1,170679.74,0,1,0
4,560,0,27,7,124995.98,1,1,1,114669.79,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,602,0,53,5,98268.84,1,0,1,45038.29,0,1,0
1996,609,1,25,10,0.00,1,0,1,109895.16,1,0,0
1997,730,0,47,7,0.00,1,1,0,33373.26,1,0,0
1998,692,1,29,4,0.00,1,1,0,76755.99,1,0,0


In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

from datetime import datetime

In [20]:
X_train.shape[1]

12

In [21]:
model = Sequential(
    [
        tf.keras.Input(shape=(X_train.shape[1],)),
        Dense(64, activation="relu", name="layer_1"),
        Dense(32, activation="relu", name="layer_2"),
        Dense(1, activation="sigmoid", name="output_layer")
    ]
)

In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ layer_1 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [24]:
log_dir="logs/fit/" + datetime.now().strftime("%m%d%Y-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [25]:
## Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [26]:
history=model.fit(X_train_scaled, y_train,
                  validation_data=(X_test_scaled, y_test),
                  validation_split=0.2,
                  epochs=100,
                  callbacks=[early_stopping_callback,tensorflow_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8090 - loss: 0.4403 - val_accuracy: 0.8245 - val_loss: 0.3926
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8386 - loss: 0.3881 - val_accuracy: 0.8525 - val_loss: 0.3553
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8586 - loss: 0.3556 - val_accuracy: 0.8585 - val_loss: 0.3408
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8584 - loss: 0.3431 - val_accuracy: 0.8585 - val_loss: 0.3435
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8608 - loss: 0.3374 - val_accuracy: 0.8595 - val_loss: 0.3433
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8619 - loss: 0.3334 - val_accuracy: 0.8475 - val_loss: 0.3717
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8637 - loss: 0.3303 - val_accuracy: 0.8625 - val_loss: 0.3389
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8627 - loss: 0.3274 - val_accu

In [27]:
model.save("artifacts/ann_churn_model.keras")

In [28]:
## Load Tensorboard Extension
%load_ext tensorboard

In [29]:
%tensorboard --logdir logs/fit

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\1.Python\das\Scripts\tensorboard.exe\__main__.py", line 2, in <module>
  File "D:\1.Python\das\Lib\site-packages\tensorboard\main.py", line 27, in <module>
    from tensorboard import default
  File "D:\1.Python\das\Lib\site-packages\tensorboard\default.py", line 30, in <module>
    import pkg_resources
ModuleNotFoundError: No module named 'pkg_resources'